In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [2]:
### define the network parameters of the adjacency matrix
N  = 1500
Kt = int(N*0.2)
J = 1/np.sqrt(N)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
J = 0.00325
ntau   = 10
trials = 30+6
tau_series = np.linspace(0,0.15,ntau)#np.linspace(0.1,0.2,ntau)#
### EEII; EEEIII; EEEIII;EEII
switch = 1
## arrays to store results
## norml0_series: norm of left eigenvector(deltaliri = 1)
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
htau = tau_series[1]-tau_series[0]
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2)) ### rank and population

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))
latent_kappa_series = np.zeros((trials,ntau,2,2))
intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population
g, gamma =  6, 1/4.0
NE = int(N/(1+gamma))
NI = N-NE#NE*gamma
N  = NE+NI ### update 
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(Kt/(1+gamma)), int(Kt/(1+gamma)*gamma) ### fixed out-degree
ce, ci = KE/NE, KI/NI
print('ce and ci:',ce,ci)
### assert that the differences between ce and ci are smaller than epsilon
epsilon = 1E-2
assert np.abs(ce-ci)<epsilon
# assert ce==ci
c = ce
ji,je = g*J,J 
### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]     = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct  = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
print('JE:',JE,'JI:',JI)

ce and ci: 0.2 0.2
eigvJ0: (-0.3899999999999837+0j)  theory: -0.39000000000000024
JE: 0.7799999999999999 JI: 1.1700000000000002


In [ ]:
# for value in {0.   0.14 0.28 0.42 0.56 0.7  0.84 0.98 1.12 1.26 1.4 }; do
#     ./run_secorder 1000 0.2 $(echo "$value*2" | bc) $value $value 1 1
# done
# for seed in {1 7 8 12 13 19 28 31 33 39 40 42 48 50 51 52 55 56 59 65 66 67 69 70 75 76 78 82 83 86 94}; do     mkdir -p "seed_$seed"; done

In [3]:
ntau   = 10#21#
trials = 30+6
tau_series = np.linspace(0,0.225,ntau)# np.linspace(0.25,0.4,ntau)#
alpha_series = (tau_series*c*(1-c)+c**2)/c**2-1
alpha_series_rec = (tau_series*2*c*(1-c)+c**2)/c**2-1
print(alpha_series)
print(alpha_series_rec)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8]


In [4]:
((alpha_series+1)*c**2-c**2)/c/(1-c)

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225])

In [6]:
# tau_series = np.array([0.025, 0.05 , 0.075, 0.1  ])
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### normalization for left and right eigenvectors 
norms_for_lvec, norms_for_rvec = np.zeros((trials,ntau+1,2)), np.zeros((trials,ntau+1,2))
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]  = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
### compute the square of the random connectivity 
Z2E = N*J**2*hat_sigmae**2*tau_series*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E, DeltaZ2I = Z2E[1]-Z2E[0],Z2I[1]-Z2I[0] ### for practical use
print('DeltaZ2E,DeltaZ2I:',DeltaZ2E,DeltaZ2I)
DeltaZ2 = np.zeros((N,N))
DeltaZ2[:,:NE], DeltaZ2[:,NE:] = DeltaZ2E, DeltaZ2I

### E population and I population separately
Z2E_E, Z2I_E = N*J**2*hat_sigmae**2*tau_series*ALPHAE, -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2E_I, Z2I_I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE, N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E_E, DeltaZ2I_E = Z2E_E[1]-Z2E_E[0],Z2I_E[1]-Z2I_E[0] ### for practical use
DeltaZ2E_I, DeltaZ2I_I = Z2E_I[1]-Z2E_I[0],Z2I_I[1]-Z2I_I[0] ### for practical use
DeltaZ2_E, DeltaZ2_I = np.zeros((N,N)),np.zeros((N,N))
DeltaZ2_E[:,:NE], DeltaZ2_E[:,NE:] = DeltaZ2E_E, DeltaZ2I_E

eigvJ0: (-1.549193338483012+0j)  theory: -1.5491933384829668
DeltaZ2E,DeltaZ2I: -0.0004000000000000003 0.0024000000000000002


### Chain motifs

In [7]:
alphachn_series  = np.array([0.000,0.100,0.200,0.300,0.400,0.500,0.600,0.700,0.800,0.900])
alpha_series_rec = np.array([0.000,0.200,0.400,0.600,0.800,1.000,1.200,1.400,1.600,1.800])

In [8]:
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_paradoxical.npz"

data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4May_PRX_60N1500_normlr_INHparadoxical.npz"#sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_paradoxical.npz"
# np.savez(data_name, **data)
### load data  
data = np.load(data_name,allow_pickle=True)
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_mean_series = data['intg_mean_series']
firing_rateeq = data['firing_rateeq']
firing_ratepert = data['firing_ratepert']

In [4]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
shiftx = 1.5
def odeIntegralP(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@(1.0+np.tanh(x-shiftx))
	return np.squeeze(dxdt)
def odesimulationP(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegralP,J=Jpt,I=I),xinit,t)

In [5]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
# Ipert[NE:]=0
Ipert[:NE]=0
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [6]:
J

0.00325

In [7]:
ce=ci=c
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.01866095 0.01859019]


In [8]:
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population


In [9]:
trials = 36
alphachns     = ['0.000','0.100','0.200','0.300','0.400','0.500','0.600','0.700','0.800','0.900']
alphachns_rec = ['0.000','0.200','0.400','0.600','0.800','1.000','1.200','1.400','1.600','1.800']
# alphachns_arr = np.array([0.1,0.2,0.3,0.4])
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
firing_ratepert = np.zeros((trials,ntau,N))
eigvchn_series = np.zeros((trials,len(alphachns),N),dtype=complex)
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))
norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))

intg_mean_series  = np.zeros((trials,ntau,N,2,3))### mean overlap matrix rank and 
# tauchns_arr = alphachns_arr*c**2/c/(1-c)
seedss = np.array([1, 4, 8, 10, 13, 14, 18, 19, 25, 26, 31, 32, 34, 36, 40, 43, 44, 49, 52, 53, 55 ,60, 62, 64, 71, 73, 76, 78, 81, 82, 83,84, 85, 86, 87, 88, 93, 94, 95])
for ktrial in range(trials):
    seed =seedss[ktrial]
    z_pre = 0
    for idxalpha, alphachn in enumerate(alphachns[:]):
        # if idxalpha <8:
        #     continue
        if idxalpha == 0:       
            file_path = 'K:\\Projects\\Allen_motif\\PRXL_data\\sonets\\N1500\\seed_'+str(seed)+'\\w_1500_0.200_'+alphachns_rec[idxalpha]+'_'+alphachn+'_'+alphachn+'_0.000_'+str(seed)+'.dat'
        else:
            file_path = 'K:\\Projects\\Allen_motif\\PRXL_data\\sonets\\N1500\\seed_'+str(seed)+'\\w_1500_0.200_'+alphachns_rec[idxalpha]+'_'+alphachn+'_'+alphachn+'_1.000_'+str(seed)+'.dat'
        Jmat = np.zeros((N,N))
        count =0
        with open(file_path, 'r') as file:
            content = file.read()
            if content == '0':
                print('File is empty') 
            else:
                for line in content.split('\n'):
                    if line:
                        Jmat[count,:]=([(x) for x in line.split()])
                        count +=1       
        Jmat[:,:NE] = Jmat[:,:NE]*je
        Jmat[:,NE:] = Jmat[:,NE:]*(-ji)

        ktrial, it = ktrial, idxalpha
        ### full rank simulation
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, Jmat, Inp)
        firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
        
        ### perturbation 
        xpert = xc_temporal[-1,:].copy()
        xpert = xpert.reshape(-1,1)
        dtt =tt[1]-tt[0]
        for ttt in range(len(tt)):
            delta_x= -xpert + Jmat@xpert.reshape(-1,1)+Ipert.reshape(-1,1)+Inp.reshape(-1,1)
            xpert = delta_x*dtt+xpert 
        firing_ratepert[ktrial,it,:] = xpert.copy().squeeze()
            
        zr = Jmat - Jbar
        ### compute the eigenvalues and eigenvectors of the matrix
        eigvchn, eigrvec = la.eig(Jmat)
        ## sort the eigenValues by it's norm of the real part 
        idxeigv = np.argsort(np.abs(eigvchn))
        idxeigv = idxeigv[::-1]
        eigvchn, eigrvec = eigvchn[idxeigv],eigrvec[:,idxeigv]
        
        eigvchn_,eiglvec = la.eig(Jmat.copy().T)
        idxeigv = np.argsort(np.abs(eigvchn_))
        idxeigv = idxeigv[::-1]
        eigvchn_, eiglvec = eigvchn[idxeigv],eiglvec[:,idxeigv]
        
        ### normalization
        reig    = np.squeeze(eigrvec[:,:].copy())
        leig0   = np.squeeze(eiglvec[:,:].copy()) 
        normval = np.sum(reig.copy()*leig0.copy(),axis=0)
        
        norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
        normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
        if np.mean(reig[:NE,0])<0:
            reig[:,0]*=-1
            leig[:,0]*=-1
            leig0[:,0]*=-1
        if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
            reig[:,1]*=-1
            leig[:,1]*=-1
            leig0[:,1]*=-1
            
        ### numerical low-rank approximation 
        ov_inp_lowrank,ov_inp_lowrank_div = np.zeros(2),np.zeros(2)
        vec_lowrank_contribution = np.zeros((N,2))
        ### linear response theory approximation 
        for i in range(2):
            ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
            ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
        Equilibrium_lowrank_outliers = np.reshape(Inp.copy(),(N,1))
        # print('EQ shape:',np.shape(Equilibrium_lowrank_outliers))
        for i in range(2):
            vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
            # print('shape :...',np.shape(np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))))
            Equilibrium_lowrank_outliers += np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))
            #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
            contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
            contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
        lowrank_eq_num[ktrial,it,:] = np.squeeze(Equilibrium_lowrank_outliers.copy())
        ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 
        
              
        print('seed:',seed,'alphachn:',alphachn,'idxalpha:',idxalpha)
        print('eigenvalues',eigvchn[:2])
        
        DELTA_Z = np.zeros((N,N))
        DELTA_Z = zr-z_pre  
        hzr = DELTA_Z.copy()
        
        eigvchn_series[ktrial,it,:] = eigvchn[:] 
        eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
        eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
        eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
        for iii in range(2):
            eiglvec0norm_series[ktrial,it,:,iii] = leig0[:,iii].copy()/normval[iii,iii]
    
        
        z_pre = zr.copy()    

C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:74: ComplexWarning: Casting complex values to real discards the imaginary part
  norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:91: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:92: ComplexWarning: Casting complex values to real discards the imaginary part
  ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:114: ComplexWarning: Casting complex values to real d

seed: 1 alphachn: 0.000 idxalpha: 0
eigenvalues [-0.3731803+0.j          0.1212225+0.08743652j]
seed: 1 alphachn: 0.100 idxalpha: 1
eigenvalues [-0.43046902+0.j         0.11947288+0.0902082j]
seed: 1 alphachn: 0.200 idxalpha: 2
eigenvalues [-0.47280494+0.j          0.14674641+0.01444523j]
seed: 1 alphachn: 0.300 idxalpha: 3
eigenvalues [-0.51062311+0.j          0.04901927-0.13757813j]
seed: 1 alphachn: 0.400 idxalpha: 4
eigenvalues [-0.56657873+0.j          0.1455212 +0.03325691j]
seed: 1 alphachn: 0.500 idxalpha: 5
eigenvalues [-0.57961959+0.j          0.15486797+0.01146191j]
seed: 1 alphachn: 0.600 idxalpha: 6
eigenvalues [-0.6151185 +0.j  0.22109177+0.j]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:76: RuntimeWarning: divide by zero encountered in divide
  leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1
C:\Users\Administrator\AppData\Local\Temp\ipykernel_39592\3874463136.py:76: RuntimeWarning: invalid value encountered in divide
  leig = leig0.copy()/normval.copy()### left eigenvector normalization to make <l|r>=1


seed: 1 alphachn: 0.700 idxalpha: 7
eigenvalues [-0.62543352+0.j  0.25386016+0.j]
seed: 1 alphachn: 0.800 idxalpha: 8
eigenvalues [-0.65686417+0.j  0.28925285+0.j]
seed: 1 alphachn: 0.900 idxalpha: 9
eigenvalues [-0.67664221+0.j  0.3148663 +0.j]
seed: 4 alphachn: 0.000 idxalpha: 0
eigenvalues [-0.3585184 +0.j  0.15132364+0.j]
seed: 4 alphachn: 0.100 idxalpha: 1
eigenvalues [-0.41146682+0.j         -0.11617187+0.09612995j]
seed: 4 alphachn: 0.200 idxalpha: 2
eigenvalues [-0.42325439+0.j          0.1364071 -0.05484101j]
seed: 4 alphachn: 0.300 idxalpha: 3
eigenvalues [-0.47089165+0.j          0.1404844 +0.06738819j]
seed: 4 alphachn: 0.400 idxalpha: 4
eigenvalues [-0.46004603+0.j          0.06647373+0.13221457j]
seed: 4 alphachn: 0.500 idxalpha: 5
eigenvalues [-0.45580698+0.j -0.15506281+0.j]
seed: 4 alphachn: 0.600 idxalpha: 6
eigenvalues [-0.46817693+0.j -0.15006938+0.j]
seed: 4 alphachn: 0.700 idxalpha: 7
eigenvalues [-0.51211278+0.j          0.11626439+0.08507349j]
seed: 4 alphachn: 

In [ ]:
# for seed in {0..36}; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed

# done

# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do
# ./run_secorder 1500 0.2 0.0 0.0 0.0 0 $seed
# ./run_secorder 1500 0.2 0.2 0.1 0.1 1 $seed
# ./run_secorder 1500 0.2 0.4 0.2 0.2 1 $seed
# ./run_secorder 1500 0.2 0.6 0.3 0.3 1 $seed
# ./run_secorder 1500 0.2 0.8 0.4 0.4 1 $seed
# ./run_secorder 1500 0.2 1.0 0.5 0.5 1 $seed
# ./run_secorder 1500 0.2 1.2 0.6 0.6 1 $seed
# ./run_secorder 1500 0.2 1.4 0.7 0.7 1 $seed
# ./run_secorder 1500 0.2 1.6 0.8 0.8 1 $seed
# ./run_secorder 1500 0.2 1.8 0.9 0.9 1 $seed
# done

# for seed in 1 4 8 10 13 14 18 19 25 26 31 32 34 36 40 43 44 49 52 53 55 60 62 64 71 73 76 78 81 82 83 84 85 86 87 88 93 94 95; do     mkdir -p "seed_$seed"; done

In [10]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [13]:
params = {'g':g,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'c':c,
          'J':J,
          'tau_series':tau_series,
          }
lst = [eigvchn_series, eigrvec_series, eiglvec_series,
       eigrvec_series_rec, eiglvec_series_rec,
        eiglvec0_series, norml0_series, params,
        lowrank_eq, firing_rateeq,lowrank_eq_num,
        ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
        contributions_lr, contributions_lr_num,
        intg_mean_series,leig0mean_series,
        norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
       "eigrvec_series_rec, eiglvec_series_rec,"
        "eiglvec0_series, norml0_series, params,"
        "lowrank_eq, firing_rateeq,lowrank_eq_num,"
        "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
        "contributions_lr, contributions_lr_num,"
        "intg_mean_series,leig0mean_series,"
        "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
data = list_to_dict(lst=lst, string=stg)
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Sparse_data/DynsStats_Adjacency_20Mar_16PRX_54N1000_normlr_whilecode.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_whilecode_used_vecrec.npz"
data_name ="E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Numerics_remote/ConnStats_WD_Adjacency_PRX_60N1500_nonISN_inh_paradoxical.npz"
np.savez(data_name, **data)
# ### load the stored data, in particular, extracting variable: eigvchn_series 
# data = np.load(data_name,allow_pickle=True)

# ### also loading other variables 
# eigvchn_series = data['eigvchn_series']
# eigrvec_series = data['eigrvec_series']
# eiglvec_series = data['eiglvec_series']
# eigrvec_series_rec = data['eigrvec_series_rec']
# eiglvec_series_rec = data['eiglvec_series_rec']
# eiglvec0_series = data['eiglvec0_series']
# norml0_series = data['norml0_series']
# params = data['params']
# intg_mean_series = data['intg_mean_series']
# leig0mean_series = data['leig0mean_series']
# # ### load data
# # data = np.load(data_name,allow_pickle=True)
# # firing_rateeq = data['firing_rateeq']
# # lowrank_eq    = data['lowrank_eq']
# # lowrank_eq_num = data['lowrank_eq_num'] 
# # contributions_lr = data['contributions_lr'] ### trial, tau, rank, pop
# # contributions_lr_num = data['contributions_lr_num']
# # eigvchn_series = data['eigvchn_series']
# # eigrvec_series = data['eigrvec_series']
# # eiglvec_series = data['eiglvec_series']
# # eiglvec0_series = data['eiglvec0_series']
# # norml0_series = data['norml0_series']
# # params = data['params']
# # intg_ov_series = data['intg_ov_series']
# # first_perturb_ov = data['first_perturb_ov']
# # first_perturb_ovP = data['first_perturb_ovP']
# # intg_mean_series = data['intg_mean_series']
# # intg_std_series = data['intg_std_series']
# # leig0mean_series = data['leig0mean_series']


In [10]:
ce, ci = c,c
radius_theo_map = np.zeros(ntau)
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0
eigvchn_theo = np.zeros((ntau,2))
eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

In [11]:
eigvchn_theo,radius_theo_map

(array([[-1.54919334, -0.        ],
        [-1.69111181,  0.14191847],
        [-1.81382715,  0.26463382],
        [-1.9235092 ,  0.37431586],
        [-2.02359627,  0.47440293],
        [-2.11623746,  0.56704412],
        [-2.20288235,  0.65368902],
        [-2.28456356,  0.73537022],
        [-2.36204746,  0.81285412],
        [-2.43592144,  0.8867281 ]]),
 array([0.56568542, 0.55136195, 0.53665631, 0.52153619, 0.50596443,
        0.48989795, 0.47328638, 0.45607017, 0.43817805, 0.41952354]))

In [13]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [22]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 8 20 21 29 24  9 15 17  4  0 30 35  3 31 14  5 13 25 28 18 10 33 34 12
 26 16  1 23  7 22] 30


In [14]:
### plot a shifting histogram for the first eigenvalue
fig,ax=plt.subplots(figsize=(6,3))
### plot the mean of the eigenvalues, how it changes with tau_series
### colormap gradiently changes with the tau_series
cm = plt.cm.YlOrRd
### the mean of the first eigenvalue
mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)
sc=ax.scatter(mean_eigvchn,-0.1*np.ones(ntau),c=tau_series,cmap=cm)
### the mean of the second eigenvalue 
cm = plt.cm.YlGn
mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)
sc=ax.scatter(mean_eigvchn,0.1*np.ones(ntau),c=tau_series,cmap=cm)

ax.set_xlabel(r'$\Re(\lambda)$',fontsize=14)
### set double ylabels
ax.set_ylabel(r'$\Im(\lambda)$',fontsize=14)
ax2 = ax.twinx() 
 
color = 'tab:gray'
ax2.tick_params(axis ='y', labelcolor = color) 
ax2.set_yticks([])
ax2.spines['bottom'].set_color('none')
### set other x-axis invisible
ax2.spines['top'].set_color('none')
ax2.spines['right'].set_color('none')
ax2.spines['left'].set_color('none')
# ax.set_title('Eigenvalue spectrum',fontsize=14,fontweight='bold')
### refine the plot 
# ax.set_xlim([-3,2])
# ax.set_ylim([-0.3,0.3])
ax.set_xlim([-4,2.5])
ax.set_ylim([-0.3,0.3])

ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
## set the x and y ticks
ax.set_xticks([-3,0,2])
ax.set_yticks([-0.5,0,0.5])
# ### also, plot the theoretical predictions for the first and second eigenvalues
# ax.scatter(eigvchn_theo[:,0].real,np.zeros(ntau),cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
# ax.scatter(eigvchn_theo[:,1].real,np.zeros(ntau),cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')

### plot a circle with the radius of the last radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[-1],color='tab:blue',fill=False,linewidth=1.5,alpha=1))
### plot a circle with the radius of the first radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[0],color='tab:purple',fill=False,linewidth=1.5,alpha=0.5,linestyle='--'))

ktrial_index = 16
### scatter plot the eigenvalue bulk of this trial
### randomly select 500 points
idx_random = np.random.choice(np.arange(2,N),size=500,replace=False)
ax.scatter(eigvchn_series[ktrial_index,0,idx_random].real,eigvchn_series[ktrial_index,0,idx_random].imag,color='tab:purple', s=5,alpha=0.15)
ax.scatter(eigvchn_series[ktrial_index,-1,idx_random].real,eigvchn_series[ktrial_index,-1,idx_random].imag,color='tab:blue', s=5,alpha=0.2)
### also plot the original eigenvalue outlier: JE-JI, using red color and X marker
ax.scatter(JE-JI,0,color='black',marker='X',s=80,alpha=1)


In [15]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',marker='o',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',marker='o')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.0,2.5])
ax.set_yticks([-3,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [33]:
### 
plt.close('all')
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)

In [18]:
mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))

thl = 1.5
ths = 1.5

for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)

            # mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            # mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            # mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            # mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6#3
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        # idxsort = np.argsort(mean_leig0vec_series[:,it,ir,0].real)
        # idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        # mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        # idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,0].real)>threshold)[0]
        # mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        
        # idxsort = np.argsort(mean_leig0vec_series[:,it,ir,1].real)
        # idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        # mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        # idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,1].real)>threshold)[0]
        # mean_leig0vec_series[idxnan,it,ir,1] = np.nan

In [20]:
dtau = tau_series[1]-tau_series[0]
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.01,0.08)
ax[0].set_yticks([0,0.05])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.45])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)

fig.tight_layout()


In [21]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,0],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$E$')
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[idx_eff,:,0,0]*eigvchn_series[idx_eff,:,0],axis=0),color='black',s=10,marker='^',label='left eigenvector '+'rank 1 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,1,0],axis=0),color='gray',s=10,label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
# ax[0].scatter(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),color='gray',s=10,marker='^',label='left eigenvector '+'rank 2 '+r'$E$')
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.2,0.35)
ax[0].set_yticks([0,0.3])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-1.5,1)
ax[1].set_yticks([-1,0])

ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5398: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[0] = start
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5399: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N + 1] = end
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5402: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\axes\_axes.py:5404: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[N+2:, 1] = dep2slice[::-1]
d:\UseApp\Anaconda\Lib\site-packages\matplotlib\cbook\__init__.py:1340: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


Text(0, 0.5, 'Mean of left \n eigenvector(I)')

In [ ]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N),facecolor='green',alpha=0.3)
## excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
# ax[0].set_ylim(-0.1,0.3)
# ax[0].set_yticks([-0.1,0.3])
ax[0].set_ylim(-0.3,0.1)
ax[0].set_yticks([-0.3,0.1])
# ax[0].set_ylim(-0.1,0.3)
# ax[0].set_yticks([-0.1,0.3])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population

# ax[1].scatter(tau_series,np.nanmean(mean_leigvec_series[idx_eff,:,0,1],axis=0),color='black',s=10,label='left eigenvector '+'rank 1 '+r'$I$')
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N),facecolor='green',alpha=0.3)
## move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.4,0.02)
ax[1].set_yticks([-0.4,0.])
# ax[1].set_ylim(-1,0.5)
# ax[1].set_yticks([-1,0.5])
ax[1].set_ylim(-0.5,1)
ax[1].set_yticks([-0.5,1])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

In [19]:
mean_rvec = np.zeros((ntau,2,2))
mean_lvec = np.zeros((ntau,2,2))

for it in range(ntau):
    for ir in range(2):
        for ip in range(2):
            mean_rvec[it,ir,ip] = np.nanmean(mean_reigvec_num_series[:,it,ir,ip])
            mean_lvec[it,ir,ip] = np.nanmean(mean_leigvec_num_series[:,it,ir,ip])*eigvchn_theo[it,ir].real
            
### compute the theoretical response function 
rank = 2
response_func_contribution = np.zeros((ntau,rank,2))

### compute the theoretical response function 
response_func = np.zeros((trials,ntau,2))
for it in range(ntau):
    response_func[:,it,1] = 1
    if it>=0:#=1:
        for ir in range(2):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)       
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
    else:
        for ir in range(1):
            response_func[:,it,0] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func[:,it,1] += (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,0] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,0]))/(1-eigvchn_theo[it,ir].real)
            response_func_contribution[it,ir,1] = (NI*(mean_lvec[it,ir,1])*(mean_rvec[it,ir,1]))/(1-eigvchn_theo[it,ir].real)

In [20]:
### 
meanfr_eq = np.zeros((trials,ntau,2))
meanfr_pert = np.zeros((trials,ntau,2))
meanfr_eq[:,:,0] = np.mean(firing_rateeq[:,:,:NE],axis=2)
meanfr_eq[:,:,1] = np.mean(firing_rateeq[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_pert[:,:,0] = np.mean(firing_ratepert[:,:,:NE],axis=2)
meanfr_pert[:,:,1] = np.mean(firing_ratepert[:,:,NE:],axis=2)
### numerical response function 
response_func_num = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
        response_func_num[ktrial,it,0]=(meanfr_pert[ktrial,it,0]-meanfr_eq[ktrial,it,0])/Ipert[-1]
        response_func_num[ktrial,it,1]=(meanfr_pert[ktrial,it,1]-meanfr_eq[ktrial,it,1])/Ipert[-1]
        
#### delete the largest 3 and the smallest 3 from response_func_num
for it in range(ntau):
    idxsort = np.where(np.abs(response_func_num[:,it,0].copy())>1e1)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,0] = np.nan 
    
    idxsort = np.where(np.abs(response_func_num[:,it,1].copy())>1e1)
    idxnan = (idxsort) 
    response_func_num[idxnan,it,1] = np.nan 
# cuts = 3 
# for it in range(ntau):
#     idxsort = np.argsort(response_func_num[:,it,0].copy())
#     idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
#     response_func_num[idxnan,it,0] = np.nan 
    
#     idxsort = np.argsort(response_func_num[:,it,1].copy())
#     idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
#     response_func_num[idxnan,it,1] = np.nan

In [22]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_mean[:,0],'k',marker='o')

ax.plot(tau_series,response_func_contribution[:,0,0],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,0],'green',marker='o',alpha=0.36)
# ### numerical using filled_between 
# ax.fill_between(tau_series,response_func_num_mean[:,0]+np.nanstd(response_func_num[:,:,:],axis=0)[:,0],response_func_num_mean[:,0]-np.nanstd(response_func_num[:,:,:],axis=0)[:,0],color='k',alpha=0.3)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,0],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,0],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('excitatory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set yaxis, ylim not too large!
ax.set_ylim([-3,6])
ax.set_yticks([-2,6])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]])

plt.show()

In [24]:
### compare response_func with response_func_num, average across the first dimension 
dtau = tau_series[1]-tau_series[0]
response_func_mean = np.nanmean(response_func[:,:,:],axis=0)
response_func_num_mean = np.nanmean(response_func_num[:,:,:],axis=0)
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(tau_series,response_func_mean[:,1],'k',marker='o')
ax.plot(tau_series,response_func_contribution[:,0,1],'orange',marker='o',alpha=0.36)
ax.plot(tau_series,response_func_contribution[:,1,1],'green',marker='o',alpha=0.36)
alphass = 0.95
ax.errorbar(tau_series[:],response_func_num_mean[:,1],yerr=np.nanstd(response_func_num[:,:,:],axis=0)[:,1],fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)

# ax.fill_between(tau_series,response_func_num_mean[:,1]+np.std(response_func_num[:,:,:],axis=0)[:,1],response_func_num_mean[:,1]-np.std(response_func_num[:,:,:],axis=0)[:,1],color='black',alpha=0.3)
# ax.plot(tau_series,response_func_num_mean[:,1],'r',marker='o')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('response function')
ax.set_title('inhibitory population')
ax.legend(['theory','numerical'])
### move x-axis at data 0, y-axis at 0 
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
## set yaxis, ylim not too large!
ax.set_ylim([-3,6])
ax.set_yticks([-2,6])
ax.set_xlim([0-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]])
plt.show()

In [24]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(5,3))

ax.plot(tau_series,eigvchn_theo[:,0],'tab:orange',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-2.5,1.5])
ax.set_yticks([-2,0,1])
ax.set_ylim([-4.5,3.5])
ax.set_yticks([-4,0,3])

ax.set_ylim([-3,1.])
ax.set_yticks([-3,0,1])
### plot y==1 
ax.plot([0,tau_series[-1]],[1,1],color='gray',linestyle='--',linewidth=1.5)
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [ ]:
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
norm_4lvec_series_select = norm_4lvec_series.copy()
norm_4lvec_series_select_ = norm_4lvec_series_.copy()
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(norm_4lvec_series[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select[idxnan,it,ir] = np.nan 
        
        idxsort = np.argsort(norm_4lvec_series_[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select_[idxnan,it,ir] = np.nan 

In [ ]:
#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:]-0.005,np.nanmean(norm_4lvec_series_select[:,1:,0]/norml0_series[:,1:-1,0],axis=0),yerr=np.nanstd(norm_4lvec_series_select[:,1:,0]/norml0_series[:,1:-1,0],axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:]+0.005,np.mean(norm_4rvec_series[:,1:,0],axis=0),yerr=np.std(norm_4rvec_series[:,1:,0],axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:],np.nanmean(norm_4lvec_series_select_[:,1:,0].real,axis=0),yerr=np.nanstd(norm_4lvec_series_select_[:,1:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:],np.mean(norm_4rvec_series[:,1:,0].real,axis=0),yerr=np.std(norm_4rvec_series[:,1:,0].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.05,tau_series[-1]+0.05])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

In [ ]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [18]:
params = {'g':g,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'J':J,
          'ce':ce,
          'ci':ci,
          'tau_series':tau_series,
          }
lst = [eigvchn_series, eigrvec_series, eiglvec_series,
       eigrvec_series_rec, eiglvec_series_rec,
        eiglvec0_series, norml0_series, params,
        lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,
        ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
        contributions_lr, contributions_lr_num,
        intg_mean_series,leig0mean_series,
        norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
       "eigrvec_series_rec, eiglvec_series_rec,"
        "eiglvec0_series, norml0_series, params,"
        "lowrank_eq, firing_rateeq,lowrank_eq_num,firing_ratepert,"
        "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
        "contributions_lr, contributions_lr_num,"
        "intg_mean_series,leig0mean_series,"
        "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4May_PRX_60N1500_normlr_INHparadoxical.npz"
data_name ="E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Numerics_remote/ConnStats_WD_Adjacency_PRX_60N1500_nonISN_inh_paradoxical.npz"
np.savez(data_name, **data)
# ### load data  
# data = np.load(data_name,allow_pickle=True)
# eigvchn_series = data['eigvchn_series']
# eigrvec_series = data['eigrvec_series']
# eiglvec_series = data['eiglvec_series']
# eiglvec0_series = data['eiglvec0_series']
# norml0_series = data['norml0_series']
# params = data['params']
# # intg_ov_series = data['intg_ov_series']
# # first_perturb_ov = data['first_perturb_ov']
# # first_perturb_ovP = data['first_perturb_ovP']
# intg_mean_series = data['intg_mean_series']
# # intg_std_series = data['intg_std_series']
# # intg_std_num_series = data['intg_std_num_series']
# # mean_shift_ov = data['mean_shift_ov']

# firing_rateeq = data['firing_rateeq']
# firing_ratepert = data['firing_ratepert']
